## BUILD THE NEURAL NETWORK

- 신경망은 데이터에 대해 작업을 수행하는 층(layer)와 모듈로 구성되어있다 
- torch.nn 은 Pytorch에서 신경망을 구성하는 다양한 레이어, 손실 함수, 활성화 함수 등과 같은 신경망과 관련된 모든 구성 요소를 포함하는 모듈이다 

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training

- GPU가 사용되지 않는 이유
    - CUDA와 GPU 드라이버 설치가 되어 있지 않는 경우
    - Pytorch GPU 버전 설치 
    - 환경 설정: GPU를 사용하기 위해 코드를 실행하는 환경이 GPU를 지원하도록 설정

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [3]:
print(torch.__version__)

2.0.1+cpu


- +cpu 로 표시되면 해당 PyTorch 버전은 CPU만 지원하는 버전

## Define the Class

- 신경망을 정의하기 위해 PyTorch에서는 nn.Module을 서브클래싱(subclassing)하는 방식을 사용한다
- 새로운 신경망 클래스를 정의할 때 nn.Module을 상속받기
- __init__ 메서드에서 신경망의 레이어들을 초기화
- forward 메서드 구현: forward 메서드는 입력 데이터에 대해 신경망의 연산을 수행하는 함수
    - 신경망은 입력 데이터를 forward 메서드에 전달하면 이 메서드에서 정의된 레이어들과 연산들을 통해 출력을 계산한다

In [4]:
class NeuralNetwork(nn.Module):

    # nn.Modlue을 사용하기 위해 super().__init__()으로 초기화
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    # (forward pass) 연산을 정의
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


- PyTorch에서는 모델에 입력 데이터를 전달하면 내부적으로 forward 메서드가 자동으로 실행되기 때문에 forward 메서드를 직접 호출할 필요가 없다
- 모델에 입력 데이터를 전달하면 모델은 순전파 연산을 수행하며, 예측 결과를 반환한다
- 반환되는 결과는 2차원 텐서로, dim=0에는 각 클래스에 대한 10개의 로우(raw) 형태의 예측 값이 저장되고, dim=1에는 각 로우의 값들이 저장된다
    -각 로우는 하나의 클래스에 해당하는 확률 값을 나타낸다
- 이러한 로우(raw) 형태의 예측 값을 확률로 변환하기 위해 nn.Softmax 모듈을 사용한다


- 모델 사용시 입력 데이터를 model(input_data)와 같이 전달하면 모델은 내부적으로 forward 메서드를 실행하여 예측 결과를 반환하고, 이후에 nn.Softmax 모듈을 사용하여 예측 결과를 확률로 변환한다


## Model Layers

- FashionMNIST 모델의 레이어를 분석해보기
- 28x28인 3개의 이미지로 구성된 샘플 미니 배치를 가져와 네트워크를 통과할 때 어떤 일이 발생하는지 확인해보기

In [7]:
# 이미지 크기 확인
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

In [8]:
# flatten으로 평탄화시킨다 28 x 28 -> 784
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

In [9]:
# 3, 784의 평탄화된 데이터를 20차원으로 변환하여 선형 변환
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0636, -0.0840,  0.0101,  0.5785,  0.0316, -0.1073,  0.3856, -0.0999,
          0.3576, -0.2465,  0.1464,  0.1077, -0.3512, -1.0581, -0.1082, -0.4951,
          0.2376, -0.0700, -0.0657,  0.1504],
        [-0.3172, -0.1456, -0.1109,  0.2341,  0.0971, -0.2446,  0.1108, -0.2669,
          0.2761, -0.1400,  0.0558,  0.1129, -0.4581, -0.9234, -0.2393, -0.4561,
         -0.2453, -0.1588, -0.3922,  0.5081],
        [-0.2455, -0.3443, -0.3474,  0.2802, -0.1236,  0.2399,  0.2598, -0.2068,
          0.1800, -0.5162,  0.3509,  0.0742, -0.3100, -0.9622, -0.0195, -0.4625,
          0.0494, -0.2617, -0.3763,  0.3353]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0101, 0.5785, 0.0316, 0.0000, 0.3856, 0.0000, 0.3576,
         0.0000, 0.1464, 0.1077, 0.0000, 0.0000, 0.0000, 0.0000, 0.2376, 0.0000,
         0.0000, 0.1504],
        [0.0000, 0.0000, 0.0000, 0.2341, 0.0971, 0.0000, 0.1108, 0.0000, 0.2761,
         0.0000, 0.0558, 0.1129, 0.0000, 0.0000, 0.00

- ReLU를 적용하기 전에는 음수와 양수 모두 포함되어있지만
- ReLU를 적용 이후에는 음수 값들이 0으로 변환되어 음수 값은 모두 제거되고, 양수 값들은 그대로 유지한다

- ReLU
    - ReLU 활성화 함수는 신경망의 비선형성을 증가시키는 역할
    - 음수 값을 0으로 제거함으로써 데이터의 특징을 강조

### nn.Sequential

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

- nn.Sequential은 여러 개의 모듈을 순차적으로 쌓아서 하나의 모듈로 만드는 데 사용


### nn.Softmax

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)
print(pred_probab.argmax(1))

tensor([[0.0936, 0.1019, 0.1204, 0.0864, 0.0791, 0.0910, 0.0921, 0.1004, 0.1053,
         0.1299],
        [0.0886, 0.1019, 0.1292, 0.0844, 0.0779, 0.0872, 0.0872, 0.0960, 0.1152,
         0.1323],
        [0.0866, 0.1009, 0.1201, 0.0936, 0.0820, 0.0992, 0.0851, 0.1076, 0.0995,
         0.1255]], grad_fn=<SoftmaxBackward0>)
tensor([9, 9, 9])


## Model Parameters

In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0042, -0.0200, -0.0299,  ...,  0.0142, -0.0069, -0.0057],
        [ 0.0315, -0.0146,  0.0063,  ..., -0.0284, -0.0290,  0.0253]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0202, 0.0021], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0441, -0.0004,  0.0019,  ...,  0.0097,  0.0181,  0.0381],
        [ 0.0195,  0.0368,  0.0350,  ...,  0.0231,  0.0359,  0.0380]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Si

In [27]:
from torchsummary import summary
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Linear(3*28*28, 20)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(20, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

model = NeuralNetwork()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 모델 요약 정보 출력
summary(model, input_size=(3, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                 [-1, 2352]               0
            Linear-2                   [-1, 20]          47,060
              ReLU-3                   [-1, 20]               0
            Linear-4                   [-1, 10]             210
Total params: 47,270
Trainable params: 47,270
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.02
Params size (MB): 0.18
Estimated Total Size (MB): 0.21
----------------------------------------------------------------


In [31]:
from torchsummary import summary
import torch.nn as nn

class NeuralNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(3*28*28, 20),
            nn.ReLU(),
            nn.Linear(20, 10),
        )
        
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
summary(model, input_size=(3, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                 [-1, 2352]               0
            Linear-2                   [-1, 20]          47,060
              ReLU-3                   [-1, 20]               0
            Linear-4                   [-1, 10]             210
Total params: 47,270
Trainable params: 47,270
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.02
Params size (MB): 0.18
Estimated Total Size (MB): 0.21
----------------------------------------------------------------


In [17]:
# 매개변수 정보 출력
print("Parameters:")
for name, param in model.named_parameters():
    print(f"{name}: {param.size()}")


Parameters:
linear_relu_stack.0.weight: torch.Size([512, 784])
linear_relu_stack.0.bias: torch.Size([512])
linear_relu_stack.2.weight: torch.Size([512, 512])
linear_relu_stack.2.bias: torch.Size([512])
linear_relu_stack.4.weight: torch.Size([10, 512])
linear_relu_stack.4.bias: torch.Size([10])


In [18]:
# 매개변수 정보 출력
print("Parameters:")
for param in model.parameters():
    print(param.size())

Parameters:
torch.Size([512, 784])
torch.Size([512])
torch.Size([512, 512])
torch.Size([512])
torch.Size([10, 512])
torch.Size([10])


In [20]:
for param in model.parameters():
    print(f"Parameter Name: {param}")
    print(f"Parameter Size: {param.size()}")
    print(f"Requires Gradient: {param.requires_grad}")
    if param.grad is not None:
        print(f"Gradient Shape: {param.grad.size()}")
    else:
        print("Gradient: None")
    print("=" * 30)

Parameter Name: Parameter containing:
tensor([[-0.0354, -0.0177, -0.0028,  ...,  0.0339,  0.0077, -0.0303],
        [-0.0107,  0.0242,  0.0285,  ...,  0.0018,  0.0073,  0.0227],
        [ 0.0018, -0.0128, -0.0081,  ...,  0.0153, -0.0113, -0.0037],
        ...,
        [-0.0221, -0.0334,  0.0128,  ..., -0.0166,  0.0340,  0.0204],
        [ 0.0179,  0.0154, -0.0149,  ..., -0.0255, -0.0023, -0.0273],
        [ 0.0312, -0.0137,  0.0276,  ..., -0.0272,  0.0164,  0.0130]],
       requires_grad=True)
Parameter Size: torch.Size([512, 784])
Requires Gradient: True
Gradient: None
Parameter Name: Parameter containing:
tensor([ 1.2099e-02,  1.8562e-02,  4.0096e-03, -2.8859e-04, -3.5697e-02,
         1.8410e-02, -3.4448e-02, -2.6702e-02, -6.1331e-03,  5.8703e-03,
        -2.5006e-02, -1.2981e-02,  3.4689e-02,  3.1626e-02,  2.7608e-02,
        -4.5151e-03, -1.5097e-02,  4.8268e-03,  9.9603e-03, -1.4238e-02,
         8.6844e-03,  9.2927e-05,  9.6027e-03, -2.9572e-02, -1.0063e-02,
         1.4774e-02,

## reference
https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html